 # Project 1: SQL Analysis and Insights 

### ****Sales Performance & Trends Analysis****:<br>

What is the trend of total revenue (gross income) and the number of orders over time (Monthly/Quarterly/Annually)?

In [194]:
import pandas as pd
from google.cloud import bigquery
from pandas_gbq import read_gbq 

project_id = 'sql-sandbox-1-450916'

In [195]:
sql_query_monthly_trend = """
SELECT
  DATE_TRUNC(O.order_purchase_timestamp, MONTH) AS month_period,
  COUNT(O.order_id) as no_Orders,
  SUM(O1.price+O1.freight_value) as Tot_revenue
FROM `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Orders` as O
JOIN `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Order_items` as O1
ON O.order_id=O1.order_id
GROUP BY month_period
ORDER BY month_period ;
"""
read_gbq(sql_query_monthly_trend, project_id=project_id)

c:\Users\Sker\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:2660: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,month_period,no_Orders,Tot_revenue
0,2016-09-01 00:00:00+00:00,6,354.75
1,2016-10-01 00:00:00+00:00,363,56808.84
2,2016-12-01 00:00:00+00:00,1,19.62
3,2017-01-01 00:00:00+00:00,955,137188.49
4,2017-02-01 00:00:00+00:00,1951,286280.62
5,2017-03-01 00:00:00+00:00,3000,432048.59
6,2017-04-01 00:00:00+00:00,2684,412422.24
7,2017-05-01 00:00:00+00:00,4136,586190.95
8,2017-06-01 00:00:00+00:00,3583,502963.04
9,2017-07-01 00:00:00+00:00,4519,584971.62


**Data Note:**
The data shows significantly lower volumes at the start (Sept-Dec 2016) and end (Sept 2018) of the dataset. This indicates the initial setup phase of data collection and the collection cutoff point, **not a business decline.**

What is the average and median value of a single delivered order monthly?

In [196]:
IndividualOrderTotals="""WITH IndividualOrderTotals AS (
    SELECT
        O.order_id,
        O.order_purchase_timestamp AS orderP_time,
        SUM(O1.price + O1.freight_value) AS total_order_value
    FROM `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Orders` AS O
    JOIN `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Order_items` AS O1
        ON O.order_id = O1.order_id
    WHERE O.order_status = 'delivered'
    GROUP BY O.order_id, orderP_time
)
SELECT
    DATE_TRUNC(orderP_time, MONTH) AS month_period,
    AVG(total_order_value) AS avg_totalOrderXmonth,
    APPROX_QUANTILES(total_order_value, 2)[OFFSET(1)] AS median_totalOrderXmonth
FROM IndividualOrderTotals
GROUP BY DATE_TRUNC(orderP_time, MONTH)
ORDER BY month_period ASC;
"""

read_gbq(IndividualOrderTotals)

c:\Users\Sker\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:2660: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,month_period,avg_totalOrderXmonth,median_totalOrderXmonth
0,2016-09-01 00:00:00+00:00,143.460000,143.46
1,2016-10-01 00:00:00+00:00,175.436453,109.02
2,2016-12-01 00:00:00+00:00,19.620000,19.62
3,2017-01-01 00:00:00+00:00,169.976493,102.90
4,2017-02-01 00:00:00+00:00,164.089123,105.83
5,2017-03-01 00:00:00+00:00,162.738001,102.92
6,2017-04-01 00:00:00+00:00,169.697091,103.63
7,2017-05-01 00:00:00+00:00,159.856571,103.37
8,2017-06-01 00:00:00+00:00,156.315907,101.25
9,2017-07-01 00:00:00+00:00,146.254928,100.77


**Insights on Average Order Value Trend:**

While the **average order value** (`avg_totalOrderXmonth`) remains relatively stable (approx. €145-€175), the **median** (`median_totalOrderXmonth`) is consistently and significantly lower (approx. €100-€110). This consistent difference points to a **right-skewed distribution** of individual order values, indicating the presence of a few high-value outlier orders pulling the mean upwards.

Therefore, the **median provides a more representative view of the *typical* order value**, reinforcing that overall revenue growth is primarily driven by an **increase in the number of orders**, rather than higher average spending per standard transaction.


### ****Logistics Analysis & Operational Efficiency****:<br>
What is the average (or median) delivery time of orders from the approval date to the customer delivery date?

In [197]:
sql_query_delivery_stats ="""
SELECT
    AVG(delivery_time_days) AS avg_daysForDel,
    APPROX_QUANTILES(delivery_time_days, 2)[OFFSET(1)] AS median_daysForDel,
    MAX(delivery_time_days) AS max_daysForDel
FROM `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Olist_Orders_Delivery_Cleaned_View`;"""

read_gbq(sql_query_delivery_stats)

c:\Users\Sker\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:2660: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,avg_daysForDel,median_daysForDel,max_daysForDel
0,11.513296,9,208


**Delivery Time Analysis:**
The average delivery time is ~11.5 days, with an outlier maximum of 208 days. However, the **median delivery time is 9 days**.
Given the highly **skewed distribution and the presence of extreme outliers**, the mean is inflated and less representative. Therefore, the **median provides a far more accurate and sensible measure of the *typical* delivery time** for the majority of orders.


What is the percentage of orders delivered "on time" compared to the estimated delivery date?


In [198]:
sql_query_on_time_delivery= """
WITH DeliveredOrdersFlagged AS (
    SELECT
        DISTINCT order_id,
        order_estimated_delivery_date,
        order_delivered_customer_date,
        CASE
            WHEN CAST(order_delivered_customer_date AS DATE) <= order_estimated_delivery_date THEN 1
            ELSE 0
        END AS on_time_orders
    FROM `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Olist_Orders_Delivery_Cleaned_View`
)
SELECT
    COUNT(DISTINCT order_id) AS tot_orders_delivered,
    SUM(on_time_orders) AS tot_orders_onTime,
    ROUND((SUM(on_time_orders) / COUNT(DISTINCT order_id)) * 100, 2) AS percent_ordrs_onTime
FROM DeliveredOrdersFlagged;
"""

read_gbq(sql_query_on_time_delivery)

c:\Users\Sker\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:2660: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,tot_orders_delivered,tot_orders_onTime,percent_ordrs_onTime
0,95780,89400,93.34


An impressive 93.34% of delivered orders arrived on time, indicating strong logistical efficiency

What is the average number of hour for an order to be approved after purchase?

In [199]:
order_approval_times_df="""SELECT
    AVG(DATETIME_DIFF(order_approved_at, order_purchase_timestamp, HOUR)) AS avg_hours_to_approve,
    APPROX_QUANTILES(DATETIME_DIFF(order_approved_at, order_purchase_timestamp, HOUR), 2)[OFFSET(1)] AS median_hours_to_approve,
    MAX(DATETIME_DIFF(order_approved_at, order_purchase_timestamp, HOUR)) AS max_hours_to_approve
FROM
    `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Orders`
WHERE
    order_approved_at IS NOT NULL
    AND order_purchase_timestamp IS NOT NULL
    AND DATETIME_DIFF(order_approved_at, order_purchase_timestamp, HOUR) >= 0;"""

read_gbq(order_approval_times_df)

c:\Users\Sker\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:2660: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,avg_hours_to_approve,median_hours_to_approve,max_hours_to_approve
0,10.064121,0,4509


**Order Approval Time Analysis:**
The average order approval time is ~10.06 hours, with an extreme maximum of 4509 hours. However, the **median approval time is 0 hours**.
This significant difference, particularly with a median of 0 and an exceptionally high maximum, indicates a **highly right-skewed distribution**. It means the vast majority (at least 50%) of orders are approved almost instantaneously (within the same hour of purchase). The mean is heavily influenced by a small number of outlier orders that face very long approval delays.
Therefore, the **median of 0 hours provides a far more accurate and representative measure of the *typical* and highly efficient initial order processing phase** for most transactions.


Analysis of Order Approval Delays by Status

In [200]:
sql_query_status_delays="""
SELECT
    O.order_status,
    COUNT(DISTINCT O.order_id) AS num_delayed_orders,
    AVG(DATETIME_DIFF(O.order_approved_at, O.order_purchase_timestamp, HOUR)) AS avg_delay_hours,
    APPROX_QUANTILES(DATETIME_DIFF(O.order_approved_at, O.order_purchase_timestamp, HOUR), 2)[OFFSET(1)] AS median_delay_hours,
    MAX(DATETIME_DIFF(O.order_approved_at, O.order_purchase_timestamp, HOUR)) AS max_delay_hours
FROM
    `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Orders` AS O
WHERE
    O.order_approved_at IS NOT NULL
    AND O.order_purchase_timestamp IS NOT NULL
    AND DATETIME_DIFF(O.order_approved_at, O.order_purchase_timestamp, HOUR) >= 0
    AND DATETIME_DIFF(O.order_approved_at, O.order_purchase_timestamp, HOUR) > 168 
GROUP BY
    O.order_status
ORDER BY
    num_delayed_orders DESC

"""
read_gbq(sql_query_status_delays)

c:\Users\Sker\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:2660: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,order_status,num_delayed_orders,avg_delay_hours,median_delay_hours,max_delay_hours
0,delivered,65,263.015385,257,741
1,canceled,7,415.142857,248,781
2,unavailable,3,1825.666667,780,4509
3,processing,2,1002.500000,555,1450
4,shipped,1,784.000000,784,784


 Analysis of Order Approval Delays by Status

 
**Summary:** This analysis reveals that a significant number of orders, including those ultimately **delivered** or **shipped**, experienced remarkably long approval delays (over 7 days). This suggests a potential systemic issue with the definition or recording process of the `order_approved_at` timestamp, rather than a primary bottleneck in the physical fulfillment chain.

## ****Customer Analysis & Geographical Distribution****:
Which states or cities generate the most orders? And which ones generate the most revenue?


In [201]:
df_geo_distribution="""
SELECT 
  C1.customer_state
  ,C1.customer_city
  ,COUNT(DISTINCT O.order_id) as no_orders
  ,ROUND(SUM(O1.price+O1.freight_value),2) as tot_revenue
FROM `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Order_items` as  O1 
JOIN `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Orders` as O
ON O.order_id=O1.order_id
JOIN `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Customers` as C1
ON C1.customer_id=O.customer_id
  GROUP BY C1.customer_state,C1.customer_city
  ORDER BY tot_revenue DESC , no_orders DESC 
  LIMIT 10;
"""
read_gbq(df_geo_distribution)

c:\Users\Sker\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:2660: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,customer_state,customer_city,no_orders,tot_revenue
0,SP,sao paulo,15402,2170227.12
1,RJ,rio de janeiro,6834,1154234.02
2,MG,belo horizonte,2750,416733.39
3,DF,brasilia,2116,352305.14
4,PR,curitiba,1510,244739.87
5,RS,porto alegre,1372,224064.09
6,BA,salvador,1238,216772.40
7,SP,campinas,1429,212541.70
8,SP,guarulhos,1178,163575.82
9,RJ,niteroi,845,137919.38


The analysis reveals that São Paulo (SP) state, particularly São Paulo city, is the dominant market leader in both order volume and total revenue. Rio de Janeiro (RJ) stands out as the second most --significant market. The top-performing locations are primarily major state capitals and large cities, indicating a concentration of e-commerce activity in Brazil's key urban centers.

What is the total number of unique customers and how has it grown over time?


In [202]:
df_customer_growth="""SELECT 
  EXTRACT(YEAR FROM O.order_purchase_timestamp) as year_
  ,EXTRACT(QUARTER FROM O.order_purchase_timestamp) as quarter_
  ,COUNT(DISTINCT C1.customer_unique_id) as no_customers
FROM `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Orders` as O
JOIN `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Customers` as C1
ON C1.customer_id=O.customer_id
  GROUP BY EXTRACT(YEAR FROM O.order_purchase_timestamp),EXTRACT(QUARTER FROM O.order_purchase_timestamp)
  ORDER BY year_ ASC,quarter_ ASC 
"""
read_gbq(df_customer_growth)

c:\Users\Sker\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:2660: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,year_,quarter_,no_customers
0,2016,3,4
1,2016,4,322
2,2017,1,5153
3,2017,2,9139
4,2017,3,12335
5,2017,4,17505
6,2018,1,20767
7,2018,2,19715
8,2018,3,12645
9,2018,4,4


The unique customer base showed robust and consistent quarter-over-quarter growth,
highlighting a significant platform expansion from 2017 onwards.

What is the distribution of order statuses (e.g., delivered, canceled, shipped)?


In [203]:
df_order_status_counts="""SELECT 
  order_status
  ,COUNT(*) as no_ordrsXstatus_
FROM `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Orders` 
  GROUP BY order_status
  ORDER BY no_ordrsXstatus_ DESC 
"""
read_gbq(df_order_status_counts)

c:\Users\Sker\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:2660: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,order_status,no_ordrsXstatus_
0,delivered,96478
1,shipped,1107
2,canceled,625
3,unavailable,609
4,invoiced,314
5,processing,301
6,created,5
7,approved,2


The vast majority of orders are successfully delivered, indicating strong fulfillment capabilities.
However, a notable percentage of canceled and unavailable orders suggest areas for process optimization.


### ****Seller & Specific Product Analysis :**** <br>
Which sellers have the best/worst performance in terms of number of orders and average delivery time?


In [204]:
df_sellers_low_volume="""SELECT 
S1.seller_city 
, S1.seller_state
  ,S1.seller_id
  ,COUNT(DISTINCT O1.order_id) as no_ordersXseller
  ,AVG(DATETIME_DIFF(O.order_delivered_customer_date, O.order_approved_at, DAY)) as avg_daysDeliveryXseller
FROM `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Orders` as O
JOIN `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Order_items` as  O1
ON O.order_id=O1.order_id
JOIN `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Sellers` as  S1
ON O1.seller_id=S1.seller_id
  WHERE
    O.order_status = "delivered"
    AND O.order_approved_at IS NOT NULL
    AND O.order_delivered_customer_date IS NOT NULL
  GROUP BY S1.seller_state,S1.seller_city 
  , S1.seller_id
  --ORDER BY no_ordersXseller DESC----SWITCH WHEN YOU NEED .
  ORDER BY no_ordersXseller ASC
  LIMIT 10;
"""

read_gbq(df_sellers_low_volume)


c:\Users\Sker\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:2660: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,seller_city,seller_state,seller_id,no_ordersXseller,avg_daysDeliveryXseller
0,bom jesus dos perdoes,SP,87f3e35268860433e13d577825aada95,1,50.0
1,sao caetano do sul,SP,3606774ec64634822d78dc2db6d37045,1,14.0
2,sao paulo,SP,ed49da7cadefc454d6a67de249eab240,1,7.0
3,ibitinga,SP,0336182e1b3e92f029d5354832045fdf,1,12.0
4,sao paulo,SP,c12d7463a579237ffb9248bd7cff378b,1,11.0
5,niteroi,RJ,fedaedd3ca31d56ab33e92035e4b361c,1,7.0
6,igaracu do tiete,SP,8b11118053756a9a2932f447e11fc1b5,1,11.0
7,sao paulo,SP,bac44fa8e13424950488659b5f765c41,1,7.0
8,bage,RS,4b90495391ac0bd181a80a459df1e718,1,11.0
9,catanduva,SP,f09b760d23495ac9a7e00d29b769007c,1,21.0


In [205]:

df_high_volume_slow_sellers="""SELECT
  S1.seller_id,
  S1.seller_city,
  S1.seller_state,
  COUNT(DISTINCT O1.order_id) as no_ordersXseller,
  AVG(DATETIME_DIFF(O.order_delivered_customer_date, O.order_approved_at, DAY)) as avg_daysDeliveryXseller
FROM `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Orders` as O
JOIN `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Order_items` as O1
ON O.order_id=O1.order_id
JOIN `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Sellers` as S1
ON O1.seller_id=S1.seller_id
WHERE
  O.order_status = "delivered"
  AND O.order_approved_at IS NOT NULL
  AND O.order_delivered_customer_date IS NOT NULL
GROUP BY S1.seller_id, S1.seller_city, S1.seller_state
HAVING no_ordersXseller >= 50-- Filter to include only sellers with a significant number of orders (at least 50) ---BECAUSE OVER ALL THE BEST SELLERS HAVE ONLY ONE ORDER 
--ORDER BY avg_daysDeliveryXseller ASC----SWITCH WHEN YOU NEED 
ORDER BY avg_daysDeliveryXseller DESC
LIMIT 10

"""
read_gbq(df_high_volume_slow_sellers)

c:\Users\Sker\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:2660: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,seller_id,seller_city,seller_state,no_ordersXseller,avg_daysDeliveryXseller
0,54965bbe3e4f07ae045b90b0b8541f52,foz do iguacu,PR,73,25.790123
1,5058e8c1e82653974541e83690655b4a,itaquaquecetuba,SP,62,25.168831
2,712e6ed8aa4aa1fa65dab41fed5737e4,videira,SC,77,22.752941
3,17f51e7198701186712e53a39c564617,sao paulo,SP,56,21.754098
4,7c67e1448b00f6e969d365cea6b010ab,itaquaquecetuba,SP,973,21.346125
5,a7f13822ceb966b076af67121f87b063,itaquaquecetuba,SP,73,20.952381
6,8444e55c1f13cd5c179851e5ca5ebd00,congonhal,MG,92,20.519608
7,6fd52c528dcb38be2eea044946b811f8,sao paulo,SP,67,20.285714
8,cac4c8e7b1ca6252d8f20b2fc1a2e4af,indaiatuba,SP,74,18.939024
9,b17b679f4f5ce2e03ce6968c62648246,ribeirao preto,SP,50,18.000000


Olist's top sellers primarily come from São Paulo, excelling in either high order volumes or efficient deliveries when active. Conversely, challenges lie with numerous inactive sellers and a subset of high-volume sellers experiencing significantly slow delivery times.


Which products or categories have the highest/lowest average review scores?

In [206]:
df_category_review_scores="""SELECT 
  P1.product_category_name
  ,AVG(R1.review_score) as avg_reviewScoreXprodct
FROM `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Products` as  P1
JOIN  `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Order_items` as  O1 ON P1.product_id=O1.product_id
JOIN  `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Reviews` as  R1 ON O1.order_id=R1.order_id
  GROUP BY P1.product_category_name
  HAVING COUNT(R1.review_score) >= 50-- Filters for categories with at least 50 reviews to ensure a more reliable average.
  ORDER BY avg_reviewScoreXprodct DESC
"""
read_gbq(df_category_review_scores)

c:\Users\Sker\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:2660: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,product_category_name,avg_reviewScoreXprodct
0,books_general_interest,4.446266
1,construction_tools_tools,4.444444
2,imported_books,4.400000
3,technical_books,4.368421
4,food_drinks,4.315412
5,luggage_accessories,4.315257
6,home_oven_coffee_portables,4.302632
7,fashion_shoes,4.233716
8,food,4.218182
9,cine_photo,4.205479


Our review analysis shows high customer satisfaction for categories like books and construction tools. Conversely, office furniture, fashion apparel, and fixed telephony categories consistently receive the lowest average scores, highlighting critical areas for Olist to address quality and customer experience.

Top 10 Macro-Categories by Revenue and Order Count :



In [207]:
TopMacroCategoriesForMetrics="""WITH TopMacroCategoriesForMetrics AS (
    SELECT
        pmc.macro_category,
        COUNT(O.order_id) AS no_Orders_macro_category,
        SUM(OI.price + OI.freight_value) AS Tot_revenue_macro_category
    FROM
        `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Order_items` AS OI
    JOIN
        `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Orders` AS O
        ON OI.order_id = O.order_id -- Join to Orders to filter out 'canceled'/'unavailable' if needed, and to count distinct orders
    JOIN
        `sql-sandbox-1-450916.Project_1_Braz_E_commerce.Olist_Products_with_MacroCategory_View` AS pmc
        ON OI.product_id = pmc.product_id
    WHERE
        O.order_status NOT IN ('canceled', 'unavailable') -- Utile per escludere ordini non finalizzati
    GROUP BY
        pmc.macro_category
)
SELECT
    macro_category,
    no_Orders_macro_category,
    Tot_revenue_macro_category,
    RANK() OVER(ORDER BY Tot_revenue_macro_category DESC) AS rank_by_revenue,
    RANK() OVER(ORDER BY no_Orders_macro_category DESC) AS rank_by_orders
FROM
    TopMacroCategoriesForMetrics
ORDER BY
    rank_by_revenue ASC
LIMIT 10;

"""
read_gbq(TopMacroCategoriesForMetrics)

c:\Users\Sker\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:2660: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,macro_category,no_Orders_macro_category,Tot_revenue_macro_category,rank_by_revenue,rank_by_orders
0,"Home, Furniture & Garden",35430,4446482.59,1,1
1,Electronics & Tech,18208,2463235.52,2,2
2,Health & Beauty,13074,1890347.50,3,3
3,Miscellaneous & Services,11034,1646825.37,4,4
4,Sports & Leisure,9259,1353265.95,5,5
5,Jewelry & Gifts,5970,1298292.47,6,7
6,Kids & Toys,7126,1035103.29,7,6
7,Automotive,4204,678606.64,8,8
8,Fashion & Apparel,3721,412826.84,9,9
9,Construction & Tools,1755,304827.52,10,10


Home, Furniture & Garden dominates, with Electronics & Tech and Health & Beauty as strong seconds.
 Macro-categories effectively aggregate top sub-category performances for strategic insights.
